***Busqueda de N1 min y max***
$$\int_\Omega \nabla u \cdot \nabla v~dx = \int_\Omega fv~dx$$
$$B_{max} = \frac{1}{2}\frac{\int V_L(t)dt}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$
$$\frac{1}{2}\frac{V_dDT_s}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$

Con

$$R = \frac{l_e}{\mu_0 \mu_e A}$$

Desmos: https://www.desmos.com/calculator/x80nw1l7ab

In [8]:
import numpy as np
import pandas as pd

def save_turns(Vd, Vo2, Vo3, D_mean, N1, Al, A_min, Bsat, Ts, dataframe):
    L1 = Al*N1**2
    N2 = N1*(Vo2/Vd)*(1-D_mean)/(D_mean)
    L2 = Al*N2**2
    N3 = N1*Vo3/Vd*(1-D_mean)/(D_mean)
    L3 = Al*N3**2

    N2_norm = np.round(N2)
    L2_norm = Al*N2**2
    N3_norm = np.round(N3)
    L3_norm = Al*N3**2

    Vo2_norm = Vd*(N2_norm/N1)*(D_mean/(1-D_mean))
    Vo3_norm = Vd*(N3_norm/N1)*(D_mean/(1-D_mean))

    Idc_max = (A_min/(N1*Al))*(Bsat - 0.5*Vd*D_mean*Ts/(N1*A_min))

    #Verify transformer space
    aproved_set = verify_space(Idc_max, N1, N2, N3, 1/Ts)
    if len(aproved_set) >= 1:
        D1 = aproved_set[0]
        D2 = aproved_set[1]
        D3 = aproved_set[2]
        Area_tot = aproved_set[3]
        J1 = aproved_set[4]
        J2 = aproved_set[5]
        J3 = aproved_set[6]
    else:
        D1 = "-"
        D2 = "-"
        D3 = "-"
        Area_tot = "-"
        J1 = "-"
        J2 = "-"
        J3 = "-"
    # add row to the DataFrame
    dataframe.loc[len(dataframe)] = [Vd, Vo2, Vo3, Al*1E9, N1, L1*1E6, N2, L2*1E6, N3, L3*1E6, ' ', Idc_max, N2_norm, L2_norm*1E6, N3_norm, L3_norm*1E6, Vo2_norm, Vo3_norm,
                                     " ", D1, D2, D3, Area_tot, J1, J2, J3]
    return dataframe


def get_N1_minmax(Idc, A_min, Vd, D, Ts, R, Bsat):
    #Solve Quadratic: 0.5*Vd*D*Ts/(A_min) + (N**2)*IDC/(R*A_min) - Bsat*N = 0
    coeffs = [Idc/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
    roots = np.roots(coeffs)
    return roots

def verify_space(Idc_max, N1, N2, N3, Fsw):
    I1_max = Idc_max
    I2_max = I1_max*N1/N2
    I3_max = 100E-3         #Guess for just control and MOS trigger, it's DC avg

    height = 27.1
    width = 8
    area_true = width*height

    D_list = [0.35, 0.6]
    SD = 66/np.sqrt(Fsw)
    J = 2
    Aw = 172
    Kw = 0.5
    K1 = 0.4
    K2 = 0.4
    K3 = 0.3

    aproved_set = []
    for D1 in D_list:
        for D2 in D_list:
            #Primary
            Scu1 = np.pi*(D1/2)**2
            Aeff1 = np.pi*(D1*SD-SD**2)
            J1 = I1_max/Aeff1
            A1 = Scu1*N1/K1
            #Secondary
            Scu2 = np.pi*(D2/2)**2
            Aeff2 = np.pi*(D2*SD-SD**2)
            J2 = I2_max/Aeff2
            A2 = Scu2*N2/K2
            #Terciary (?)
            D3 = min(D_list)
            Scu3 = np.pi*(D3/2)**2
            Aeff3 = np.pi*(D3*SD-SD**2)
            J3 = I3_max/Aeff3
            A3 = Scu3*N1/K3

            Area_tot = A1 + A2 + A3

            #Aproval condition:
            #if (Area_tot < 0.9*area_true) and (J1<=2) and (J2<=2) and (J3<=2):
            aproved_set.append([D1, D2, D3, Area_tot, J1, J2, J3])
    if len(aproved_set) >= 1:
        aproved_set = aproved_set[len(aproved_set)-1]       #Keep biggest aproved result

    return aproved_set

#Datos
Vd = 30
Vo2 = 115
Vo3 = 18
Bsat = 220E-3   #En realidad la datasheet dice 320mT pero Miguel dice que mas de 220mT no le sacamos
#Ponemos un D alto para asegurarnos que ni asi va a saturar
D_max = 0.45    #Maximo conservador para el modulo pwm
D_mean = 0.35
Fs = 75E3
Ts = 1/Fs
#Corriente DC maxima para el MOS
Idc = 2
#Datos del nucleo
Al = 5200E-9
A_min = 229E-6
le = 97E-3
u0 = 4*np.pi*1E-7
ue = 1690
Ae = 234E-6
R = le/(u0*ue*A_min)
#Si usamos un gap
lgap_list = [0.2E-3, 0.4E-3, 0.6E-3] #Paper gaps

for lgap in lgap_list:
    print("***** lgap = ", lgap*1e3, "mm *****", end="")
    R = lgap/(u0*Ae)
    Al = 1/R
    valid_N1_roots = []
    for Idc in range(100, 1500, 20):
        roots = get_N1_minmax(Idc/1000, A_min, Vd, D_max, Ts, R, Bsat)
        #print(roots)
        #Only consider currents that yield real turns, then only consider those that are further apart than 10 turns and closer than 30
        if type(roots[0]) != type(np.complex128(1+1j)):
            if (max(roots) - min(roots) >= 5) and (max(roots) - min(roots) <= 50):
                #print("\tIdc = ", Idc/1000, "A. N1: ", roots)
                valid_N1_roots.append(roots)

    #Keep the latest valid N1 set, as that is the highest current

    N1_range = valid_N1_roots[len(valid_N1_roots)-1]
    print(" N1_range = ", np.round(N1_range, decimals=2), " *****")
    #Analyze each possible range of N1
    # create an empty DataFrame
    df = pd.DataFrame(columns=['Vd', 'Vo2', 'Vo3', 'AL[nHy]', 'N1', 'L1[uHy]', 'N2', 'L2[uHy]', 'N3', 'L3[uHy]', 'Normalizamos...', 'Idc_max[A]', 'N2_norm', 'L2_norm[uHy]', 'N3_norm', 'L3_norm[uHy]', 'Vo2_norm', 'Vo3_norm',
                               'Window->', 'D1', 'D2', 'D3', 'Area Total', 'J1', 'J2', 'J3'])
    
    N1_min = int(np.ceil(min(N1_range)))
    N1_max = int(np.floor(max(N1_range)))
    
    for N1 in range(N1_min, N1_max):
        # add some rows to the DataFrame
        df = save_turns(Vd, Vo2, Vo3, 0.30, N1, Al, A_min, Bsat, Ts, df)
    print(df.to_string(index=False, max_colwidth=20, float_format='%.2f'))
    print("")

***** lgap =  0.2 mm ***** N1_range =  [21.2   1.95]  *****
Vd Vo2 Vo3  AL[nHy] N1  L1[uHy]     N2  L2[uHy]    N3  L3[uHy] Normalizamos...  Idc_max[A]  N2_norm  L2_norm[uHy]  N3_norm  L3_norm[uHy]  Vo2_norm  Vo3_norm Window->   D1   D2   D3  Area Total    J1   J2   J3
30 115  18  1470.27  2     5.88  17.89   470.50  2.80    11.53                        6.93    18.00        470.50     3.00         11.53    115.71     19.29          0.60 0.60 0.35       14.70 25.50 2.85 1.21
30 115  18  1470.27  3    13.23  26.83  1058.63  4.20    25.94                        6.89    27.00       1058.63     4.00         25.94    115.71     17.14          0.60 0.60 0.35       22.05 25.34 2.83 1.21
30 115  18  1470.27  4    23.52  35.78  1882.01  5.60    46.11                        6.02    36.00       1882.01     6.00         46.11    115.71     19.29          0.60 0.60 0.35       29.40 22.13 2.47 1.21
30 115  18  1470.27  5    36.76  44.72  2940.64  7.00    72.04                        5.22    45.00     

IndexError: list index out of range